In [101]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from sklearn import linear_model
%matplotlib inline

In [121]:
user_info = pd.read_csv("data/user-features")
question_info = pd.read_csv("data/question-features")
train_info = pd.read_csv("data/invited_info_train.txt", sep="\t", header=None, names=[
    "question_id", "user_id", "answered"
])
test_info = pd.read_csv("data/validate_nolabel.txt", sep=",", names=[
    "question_id", "user_id", "answered"
])

TRAIN_DATA = train_info.merge(question_info, left_on='question_id', right_on='question_id', how='inner')
TRAIN_DATA = TRAIN_DATA.merge(user_info, left_on='user_id', right_on='user_id', how='inner')

TEST_DATA = test_info.merge(question_info, left_on='question_id', right_on='question_id', how='inner')
TEST_DATA = TEST_DATA.merge(user_info, left_on='user_id', right_on='user_id', how='inner')

FEATURES = [ 
#   "tag", "upvotes", "answers", "top_answers",'answerability', 
  'nTag', 'votability', 'ease', 'popularity', 'answerability', 
  'wq1', 'wq2', 'wq3', 'wq4', 'wq5', 'wq6', 'wq7', 'wq8', 'wq9', 'wq10', 
  'cq1', 'cq2','cq3', 'cq4', 'cq5', 'cq6', 'cq7', 'cq8', 'cq9', 'cq10', 
  't1', 't2', 't3', 't4', 't5', 
  'w1', 'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8', 'w9', 'w10', 
  'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10'
]

TARGET = 'answered'

In [118]:
model = linear_model.LogisticRegression(
    penalty='l2', 
    dual=False, 
    tol=0.0001, 
    C=1.0, 
    fit_intercept=True, 
    intercept_scaling=1, 
    class_weight='balanced', 
    random_state=None, 
    solver='liblinear', 
    max_iter=1000, 
    multi_class='ovr', 
    verbose=0, 
    warm_start=False, 
    n_jobs=1
)
model.fit(TRAIN_DATA[FEATURES], TRAIN_DATA[TARGET])
predictions = model.predict_proba(TEST_DATA[FEATURES])

In [119]:
res = TEST_DATA[['question_id', 'user_id']].copy()
res['prediction'] = predictions[:,1]
res.columns = ['qid','uid','label']
res.to_csv('results.csv', sep=",", index=None)